In [1]:
grammar = """S -> NP VP
S -> VP
NP -> DT NN
NP -> IN NNP
NP -> DT JJ NN
NP -> PRP
VP -> VBP NP
VP -> VBP VP
VP -> VBG NP
VP -> TO VP
VP -> VB
VP -> VB NP
VP -> VBD
VP -> VBD NP
VBP -> 'am'
NNP -> 'paul'
PRP -> 'i'
IN -> 'with'
VBG -> 'talking'"""

In [2]:
import re

productions = grammar.split('\n')
pos_table = []

for production in productions:
    nonterminal, sequence = production.split(' -> ')

    symbols = re.sub('\'', '', sequence).split()
    pos_table.append((nonterminal, symbols))

pos_table

[('S', ['NP', 'VP']),
 ('S', ['VP']),
 ('NP', ['DT', 'NN']),
 ('NP', ['IN', 'NNP']),
 ('NP', ['DT', 'JJ', 'NN']),
 ('NP', ['PRP']),
 ('VP', ['VBP', 'NP']),
 ('VP', ['VBP', 'VP']),
 ('VP', ['VBG', 'NP']),
 ('VP', ['TO', 'VP']),
 ('VP', ['VB']),
 ('VP', ['VB', 'NP']),
 ('VP', ['VBD']),
 ('VP', ['VBD', 'NP']),
 ('VBP', ['am']),
 ('NNP', ['paul']),
 ('PRP', ['i']),
 ('IN', ['with']),
 ('VBG', ['talking'])]

Add auxiliary terminals in order to obtain only two nodes in each production:

In [3]:
ct = 1
new_terminals = []

for i in range(len(pos_table)):
    while len(pos_table[i][1]) > 2:
        new_terminals.append(('T' + str(ct), pos_table[i][1][:2]))
        pos_table[i] = (pos_table[i][0], ['T' + str(ct)] + pos_table[i][1][2:])
        ct += 1

pos_table += new_terminals
pos_table

[('S', ['NP', 'VP']),
 ('S', ['VP']),
 ('NP', ['DT', 'NN']),
 ('NP', ['IN', 'NNP']),
 ('NP', ['T1', 'NN']),
 ('NP', ['PRP']),
 ('VP', ['VBP', 'NP']),
 ('VP', ['VBP', 'VP']),
 ('VP', ['VBG', 'NP']),
 ('VP', ['TO', 'VP']),
 ('VP', ['VB']),
 ('VP', ['VB', 'NP']),
 ('VP', ['VBD']),
 ('VP', ['VBD', 'NP']),
 ('VBP', ['am']),
 ('NNP', ['paul']),
 ('PRP', ['i']),
 ('IN', ['with']),
 ('VBG', ['talking']),
 ('T1', ['DT', 'JJ'])]

In [4]:
def check_symbols(symbol_list):
    for production in pos_table:
        if production[1] == symbol_list:
            return production[0]

    return None

check_symbols(['VB', 'NP'])

'VP'

In [5]:
import numpy as np
import nltk.data
import string

def posTagSentence(sentence):
    sentence_to_array = nltk.wordpunct_tokenize(sentence)
    sentence_with_pos = nltk.pos_tag(sentence_to_array)
    sentence_without_punct = [(word[1], word[0].lower()) for word in sentence_with_pos if word[0][0] not in string.punctuation]
    return sentence_without_punct

posTagSentence('I am talking with Paul.')

[('PRP', 'i'),
 ('VBP', 'am'),
 ('VBG', 'talking'),
 ('IN', 'with'),
 ('NNP', 'paul')]

In [35]:
def get_production_depth(productions, sequence, pos, current_pos):
    if productions == None:
        productions = []

    nonterminal = check_symbols(sequence)

    while nonterminal:
        productions.append((nonterminal, sequence, pos))
        pos = current_pos
        sequence = [nonterminal]
        nonterminal = check_symbols(sequence)

    return productions if productions != [] else None

In [36]:
import itertools

def all_combinations(list1, list2, pos):
    if list1 == None or list2 == None:
        return []

    combinations = list(itertools.chain(*[
        list(zip(each_permutation, list2)) for each_permutation in itertools.permutations(list1, len(list2))
    ] + ([
        list(zip(list1, each_permutation)) for each_permutation in itertools.permutations(list2, len(list1))
    ] if len(list1) != len(list2) else [])))
    
    return [(c[0], c[1], pos) for c in combinations]

terminals = all_combinations([('PRP', 'i'), ('NP', ['PRP'])], [('IN', 'with')], [(1, 1), (2, 2)])
terminals

[(('PRP', 'i'), ('IN', 'with'), [(1, 1), (2, 2)]),
 (('NP', ['PRP']), ('IN', 'with'), [(1, 1), (2, 2)])]

In [45]:
import numpy as np

def make_parsing_matrix(sentence_string):
    sentence = posTagSentence(sentence_string)
    n = len(sentence)
    T = np.full((n, n), None)

    for i in range(n):
        T[i][i] = get_production_depth([sentence[i]], [sentence[i][0]], [(i, i)], (i, i))

    for j in range(1, n):
        for i in range(n - j):
            terminals = list(itertools.chain(*[
                all_combinations(T[i][k], T[k + 1][i + j], [(i, k), (k + 1, i + j)]) for k in range(i, i + j)
            ]))
            for terminal1, terminal2, pos in terminals:
                T[i][i + j] = get_production_depth(T[i][i + j], [terminal1[0], terminal2[0]], pos, [(i, i + j)])

    return T

T = make_parsing_matrix('I am talking with Paul.')
T

array([[list([('PRP', 'i'), ('NP', ['PRP'], [(0, 0)])]), None, None,
        None, list([('S', ['NP', 'VP'], [(0, 0), (1, 4)])])],
       [None, list([('VBP', 'am')]), None, None,
        list([('VP', ['VBP', 'VP'], [(1, 1), (2, 4)]), ('S', ['VP'], [(1, 4)])])],
       [None, None, list([('VBG', 'talking')]), None,
        list([('VP', ['VBG', 'NP'], [(2, 2), (3, 4)]), ('S', ['VP'], [(2, 4)])])],
       [None, None, None, list([('IN', 'with')]),
        list([('NP', ['IN', 'NNP'], [(3, 3), (4, 4)])])],
       [None, None, None, None, list([('NNP', 'paul')])]], dtype=object)

In [78]:
def get_parsing(T, terminal, pos):
    productions = T[pos[0]][pos[1]]
    i = 0
    while i < len(productions) and productions[i][0] != terminal:
        i += 1

    productions = productions[i]
    if productions[1][0].lower() == productions[1][0]:
        return ' (' + terminal + ' ' + productions[1] + ')'

    return ' (' + terminal + \
            get_parsing(T, productions[1][0], productions[2][0]) + \
            (get_parsing(T, productions[1][1], productions[2][1]) if len(productions[1]) > 1 else '') + \
    ')'

get_parsing(T, 'S', (0, T.shape[1] - 1))

' (S (NP (PRP i)) (VP (VBP am) (VP (VBG talking) (NP (IN with) (NNP paul)))))'

In [79]:
def find_sentence_parsing(sentence):
    T = make_parsing_matrix(sentence)
    parsing = get_parsing(T, 'S', (0, T.shape[1] - 1))
    return parsing

print(find_sentence_parsing('I am talking with Paul.'))

 (S (NP (PRP i)) (VP (VBP am) (VP (VBG talking) (NP (IN with) (NNP paul)))))
